In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/unlabeled-articles/unlabeled_articles/114861.json
/kaggle/input/unlabeled-articles/unlabeled_articles/117240.json
/kaggle/input/unlabeled-articles/unlabeled_articles/119467.json
/kaggle/input/unlabeled-articles/unlabeled_articles/113517.json
/kaggle/input/unlabeled-articles/unlabeled_articles/113580.json
/kaggle/input/unlabeled-articles/unlabeled_articles/119654.json
/kaggle/input/unlabeled-articles/unlabeled_articles/113583.json
/kaggle/input/unlabeled-articles/unlabeled_articles/116079.json
/kaggle/input/unlabeled-articles/unlabeled_articles/119295.json
/kaggle/input/unlabeled-articles/unlabeled_articles/116502.json
/kaggle/input/unlabeled-articles/unlabeled_articles/115841.json
/kaggle/input/unlabeled-articles/unlabeled_articles/117872.json
/kaggle/input/unlabeled-articles/unlabeled_articles/116769.json
/kaggle/input/unlabeled-articles/unlabeled_articles/113351.json
/kaggle/input/unlabeled-articles/unlabeled_articles/114549.json
/kaggle/input/unlabeled-articles/unlabel

In [2]:
# 載入模型
import os, json, torch
from transformers import BertTokenizerFast, BertForSequenceClassification
from tqdm import tqdm

# 類別映射
label_map = {
    0: "賽事戰報",
    1: "球隊分析",
    2: "球員焦點",
    3: "交易與簽約",
    4: "教練與管理層",
    5: "選秀觀察",
    6: "歷史與專題"
}

# 載入 tokenizer 與模型
!unzip -q /kaggle/input/sportsv/basketball-bert.zip
!ls ./basketball-bert
tokenizer = BertTokenizerFast.from_pretrained("./basketball-bert")
model = BertForSequenceClassification.from_pretrained("./basketball-bert")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# 預測函式
def predict(text):
    encoding = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    encoding = {k: v.to(device) for k, v in encoding.items()}
    with torch.no_grad():
        outputs = model(**encoding)
        pred = torch.argmax(outputs.logits, dim=1).item()
    return label_map[pred]

# ✅ 輸出資料夾
os.makedirs("./predicted_articles", exist_ok=True)

count = 0
for fname in tqdm(os.listdir("/kaggle/input/unlabeled-articles/unlabeled_articles")):
    if not fname.endswith(".json"):
        continue
    try:
        with open(f"/kaggle/input/unlabeled-articles/unlabeled_articles/{fname}", "r", encoding="utf-8") as f:
            data = json.load(f)
            content = " ".join(data.get("article-content", []))
            if not content.strip():
                continue
            prediction = predict(content)
            data["predicted_category"] = prediction
        with open(f"./predicted_articles/{fname}", "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        count += 1
    except Exception as e:
        print(f"❌ {fname} 發生錯誤：{e}")
print(f"✅ 預測完成，共處理 {count} 篇")

# ✅ 壓縮輸出資料夾供下載
!zip -qr predicted_articles.zip predicted_articles

2025-06-01 08:00:12.270779: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748764812.445031      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748764812.498162      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json	   special_tokens_map.json  tokenizer.json
model.safetensors  tokenizer_config.json    vocab.txt


100%|██████████| 1161/1161 [00:29<00:00, 39.12it/s]

✅ 預測完成，共處理 1161 篇



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
